Forma Schur Reală
================

$\qquad$Etapa iterativă a algoritmului QR utilizează, într-o manieră implicită, metodele puterii și puterii inverse pentru reducerea unei matrice la forma Schur (reală).

$\qquad$In vederea minimizării efortului de calcul, într-o fază preliminară, matricea dată este adusă, prin transformări de asemănare ce implică un număr (teoretic) finit si (practic) rezonabil de mic de operații, la cea mai apropiată structură posibilă de forma Schur (reală). Această structură este forma superior Hessenberg. În continuare, structura Hessenberg este conservată de recurență fazei iterative a algoritmului. În acest fel, se obține o importantă reducere a complexității unei iterații **QR**, fapt esențial în economia algoritmului.

$\qquad$*ALGORITM(Calculul iterativ al formei Schur reale)*(Dată o matrice $A \in \mathbb{R}^{nxn}$ cu spectru real și o tolerantă $tol$ pentru anularea elementelor subdiagonale, algoritmul calculează matricea superior Hessenberg $H = Q^TAQ$  ortogonal asemenea cu A și apoi suprascrie H cu matricea $H_k$ din din șirul **QR** cu pași simpli cu deplasare explicită. Tipărește la final numărul de iterații pentru a ajunge la forma Schur reală.)

1. $[H, Q] = \textbf{HQ}(A)$
2. $iter \gets 0$
3.  $m \gets n$
4. **cât timp** $m > 1$ 
    * $\mu \gets H(m,m)$
    * **pentru** $i = 1:m,$
        + $H(i,i) \gets H(i,i) - \mu$
    * $[\tilde{Q},R] = \textbf{qr}H(1:m,1:m)$
    * $H(1:m,1:m)  \gets R\tilde{Q}$
    * **pentru**$i = 1:m$
        + $H(i,i) \gets H(i,i) + \mu$
    *  **dacă** $m < n$ **atunci**
        + $H(1:m,m+1:n) \gets \tilde{Q}^TH(1:m,m+1:n)$
    *  $Q(:,1:m) \gets Q(:,1:m)\tilde{Q}$
    * $iter \gets iter + 1$
    * **dacă** $iter > maxiter$ **atunci**
        + tipărește ’S-a atins numărul maxim de iterații fără a se fi obținut nivelul precis al toleranței.’
        + **stop**
    * **tipărește** $iter$
    * **dacă** $|H(m,m-1)| < tol$ **atunci**
        + $H(m,m-1) \gets 0$
        + $m \gets m - 1$
        
$\qquad$ Algoritmul funcționează pentru matrice dense cu cu spectru real impus astfel:

$L = [l_1, l_2, ..., l_n];\qquad$ -- unde $ l_i$ sunt valori proprii impuse , i = 1:n
<br>
$A = diag(L);$
<br>
$T = randn(n);$	
$A = T*A/T;$

In [15]:
#functia QR(algoritmul care calculeaza forma schur reala)
import numpy as np
import math
from scipy.linalg import schur
from scipy.linalg import hessenberg

def QR(A,tol,maxiter):
    n = len(A)
    B = np.copy(A)
    H, Q = hessenberg(B,calc_q=True)
    iter = 0
    m = n
    mesaj = 0
    while m > 1:
        miu = H[m-1][m-1]
        for i in range(m):
            H[i][i] = H[i][i] - miu
            
        Qtilda, R = np.linalg.qr(H[0:m,0:m])
        H[0:m,0:m] = np.dot(R,Qtilda)
    
        for i in range(m):
            H[i][i] = H[i][i] + miu
            
        if m < n:
            H[0:m,(m+1):n] = np.copy(np.dot(np.transpose(Qtilda),H[0:m,(m+1):n]))
            
        Q[:,0:m] = np.dot(Q[:,0:m],Qtilda)
        iter = iter + 1
        if iter > maxiter:
            mesaj = 1
            H = float('inf')
            return H, mesaj, iter
        
        if abs(H[m-1][m-2]) < tol:
            H[m-1][m-2] = 0
            m = m - 1
           
    return H, mesaj, iter
#--------------------------------------------------------------------------
n = 5
L = np.zeros((n))
for i in range(n):
    L[i] = np.random.randint(0, 9)
A = np.diag(L)
T = np.random.randn(n,n)
A = np.dot(np.dot(T,A),np.linalg.inv(T))

tol = 1e-10
maxiter = 100

#---------------------------------------------------------------------------
#verificare
E = np.copy(A)
H, mesaj, iteratii = QR(E,tol,maxiter)
if not mesaj:
    Hver, Qver = schur(A)
    if np.allclose(H.all(),Hver.all()):
        print("Matricea este:")
        print(H)
        print("Numarul de iterații fiind ", iteratii)
    else:
        print("Matrice calculată greșit")
else:
    print("S-a atins numărul maxim de iterații")
#---------------------------------------------------------------------------

Matricea este:
[[ 7.          0.49735089  3.75854937 -2.10680336  7.74446114]
 [ 0.          8.         -2.07236301 -1.90993285  2.66862253]
 [ 0.          0.          1.          0.21818626 -0.4470275 ]
 [ 0.          0.          0.          2.         -1.93880357]
 [ 0.          0.          0.          0.          3.        ]]
Numarul de iterații fiind  15
